In [5]:
import requests
import json
from bs4 import BeautifulSoup


def fetch_data(size):
    documents = []
    per_page = 100
    offset = 0
    # Paginated fetch
    while len(documents) < size:
        # Read from the api
        url_string = 'https://api.poliflw.nl/v0/search?size={}&from={}'.format(per_page, offset)
        request = requests.get(url_string)
        json_response = json.loads(request.text)
        # Return the array of document items
        items = json_response['item']
        for item in items:
            if len(documents) < size:
                documents.append(item)

        offset += per_page

    return documents


def fetch_latest_documents(size=1000):
    # Fetch document array
    documents = fetch_data(size)
    return documents


def fetch_single_document(article_id: str):
    # Read from the api
    url_string = 'https://api.poliflw.nl/v0/combined_index/{}'.format(article_id)
    request = requests.get(url_string, auth=(PFL_USER, PFL_PASSWORD))
    json_response = json.loads(request.text)

    # Fix that makes sure the article json contains an id.
    json_response['meta']['_id'] = article_id

    # Return article document from poliflow.
    return json_response

def html2text(html: str) -> float:
    """
    Process the html and transform it to standard text without html tags or weird symbols.
    :param html: The html as string.
    :return: The parsed html as text without any html tags.
    """
    # TODO: Does not yet successfully handle all html input like &amp;
    soup = BeautifulSoup(html, 'html.parser')
    text = soup.get_text().strip().replace('\n', ' ')
    return text

In [7]:
docs = fetch_latest_documents(10000)

In [45]:
from langdetect import detect
from langdetect import DetectorFactory
DetectorFactory.seed = 0

In [50]:
description_lengths = []
description_words = []
languages = {}

for doc in docs:
    if 'description' in doc:
        text_description = html2text(doc['description'])

        description_lengths.append(len(text_description))
        description_words.append(len(text_description.split(" ")))



        if len(text_description) > 200:
            
            try:
                lan = detect(text_description)

                if not lan in languages:
                    languages[lan] = 1
                else:
                    languages[lan] = languages[lan] + 1
            except:
                print(text_description)

/home/rothweiler/.local/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://veenendaal.groenlinks.nl/node/112106" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/rothweiler/.local/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://veenendaal.groenlinks.nl/nieuws/oud-papier-hot-veenendaal" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/rothweiler/.local/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://tweedekamer.groenlinks.nl/node/94725" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client lik

In [51]:
import numpy as np


print(np.mean(description_lengths))
print(np.mean(description_words))

print(languages)

1249.7085985652218
193.35879559462464
{'nl': 8201, 'en': 74, 'af': 14, 'de': 5, 'ca': 3, 'fr': 3, 'sv': 1}
